In [4]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import interpolate
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'

In [5]:
forecast_weather = pd.read_csv('C:\\Users\\user\\github\\Data\\capstone\\weather/solar_forecast_weather.csv')
solar = pd.read_csv('C:\\Users\\user\\github\\Data\\capstone\\weather/solar_power_2204.csv')
real_weather = pd.read_csv('../../Data/capstone/total_eda_real.csv')
solar_2 = pd.read_csv('../../Data/capstone/발전소/하동_eda.csv')

In [6]:
def time_change(data, Forecast_time):
    """
    data = forecast가 있는 데이터
    Forecast_time = 예보시간

    return = 예보시간을 기준으로 데이터를 정렬한 데이터
    """
    data[Forecast_time] = pd.to_datetime(data[Forecast_time])
    data['real_time'] = data.apply(lambda row: row['Forecast time'] + datetime.timedelta(hours=row['forecast']), axis=1)
    forecast_weather_eda = data.groupby(['real_time']).mean().drop('forecast', axis=1)
    forecast_weather_eda.reset_index(inplace=True)
    return forecast_weather_eda

In [7]:
eda_forecast = time_change(forecast_weather, 'Forecast time')

C:\Users\user\AppData\Local\Temp\ipykernel_5760\2365867269.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  forecast_weather_eda = data.groupby(['real_time']).mean().drop('forecast', axis=1)


In [8]:
solar['datetime'] = pd.to_datetime(solar['datetime'])
solar.target.replace(65535.0, solar.target.mean(), inplace=True)
solar['month'] = solar.datetime.dt.month
solar['hour'] = solar.datetime.dt.hour
solar['year'] = solar.datetime.dt.year
solar['day'] = solar.datetime.dt.day
solar_eda = solar.groupby(['year','month','day','hour']).mean().reset_index()
solar_eda['date'] = pd.to_datetime(solar_eda[['year', 'month', 'day', 'hour']])

C:\Users\user\AppData\Local\Temp\ipykernel_5760\108906182.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  solar_eda = solar.groupby(['year','month','day','hour']).mean().reset_index()


In [9]:
incheon = real_weather[real_weather['지점명'] =='인천']

eda_weather = incheon.drop(['지점', '지점명'], axis=1)

In [10]:
eda_weather['일시'] = pd.to_datetime(eda_weather['일시'])

In [11]:
eda_weather['hour'] = eda_weather['일시'].dt.hour
eda_weather['month'] = eda_weather['일시'].dt.month
eda_weather['year'] = eda_weather['일시'].dt.year

In [12]:
eda_weather.set_index('일시', inplace=True)

In [13]:
solar_eda.set_index('date', inplace=True)

In [14]:
total_time = pd.date_range(start='2021-01-01 00:00:00', end='2022-04-30 23:00:00', freq='H')

In [15]:
solar_2.date = pd.to_datetime(solar_2.date)

In [16]:
solar_2.set_index('date', inplace=True)

In [17]:
y = solar_2.loc[total_time].value

In [18]:
eda_weather.drop(['위도', '경도'], axis=1, inplace=True)

In [19]:
x = eda_weather.loc['2021-01-01 00:00:00' : '2022-04-30 23:00:00']

In [20]:
comm = y.index.intersection(x.index)
comm_2 = x.index.intersection(y.index)

In [21]:
x = x.loc[comm]
y = y.loc[comm]

x = x.loc[comm_2]
y = y.loc[comm_2]

In [22]:
y_input = y.loc[('2022-01-14 00:00:00' > y.index) | (y.index > '2022-01-17 23:00:00')]

In [23]:
x_input = x.loc[('2022-01-14 00:00:00' > x.index) | (x.index > '2022-01-17 23:00:00')]

In [24]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [25]:
sc = StandardScaler()

In [26]:
slice = 8760

In [27]:
x_train = x_input.iloc[:slice]
x_test = x_input.iloc[slice:]
y_train = y_input.iloc[:slice]
y_test = y_input.iloc[slice:]

In [28]:
sc.fit(x_train)

StandardScaler()

In [29]:
sc_x_train = sc.transform(x_train)
sc_x_test = sc.transform(x_test)

In [30]:
sc.fit(y_train.values.reshape(-1,1))

StandardScaler()

In [31]:
sc_y_train = sc.transform(y_train.values.reshape(-1,1)).reshape(-1)
sc_y_test = sc.transform(y_test.values.reshape(-1,1))

In [32]:
print(len(sc_x_train), len(sc_x_test), len(sc_y_train), len(sc_y_test))

8760 2783 8760 2783


In [33]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
import tensorflow as tf

In [43]:
tf_x_train = tf.convert_to_tensor(sc_x_train)
tf_x_test = tf.convert_to_tensor(sc_x_test)
tf_y_train = tf.convert_to_tensor(sc_y_train)
tf_y_test = tf.convert_to_tensor(sc_y_test)

In [45]:
tf_x_train

<tf.Tensor: shape=(8760, 8), dtype=float64, numpy=
array([[-2.00116324, -0.57071197, -1.76660599, ..., -1.66132477,
        -1.60274527,  0.        ],
       [-2.02988294, -0.63486077, -1.48508198, ..., -1.51686175,
        -1.60274527,  0.        ],
       [-2.02988294, -1.21219999, -1.76660599, ..., -1.37239873,
        -1.60274527,  0.        ],
       ...,
       [-1.92457737,  0.96885927,  1.23631674, ...,  1.37239873,
         1.58764752,  0.        ],
       [-1.97244354,  0.19907365,  1.42399941, ...,  1.51686175,
         1.58764752,  0.        ],
       [-1.98201677,  0.45566886, -0.64050996, ...,  1.66132477,
         1.58764752,  0.        ]])>

In [49]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(8760, ), return_sequences=True))
lstm_model.add(LSTM(64))
lstm_model.add(Dense(1))
lstm_model.compile(loss='mse', optimizer='adam', metrics=['mse'])

ValueError: Input 0 of layer "lstm_8" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 8760)

In [50]:
lstm_model.fit(tf_x_train, tf_y_train, epochs=100, batch_size=1, verbose=1)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.